In [ ]:
#!/usr/bin/python

import http.client as httplib
import httplib2
import os
import random
import sys
import time
import argparse
import glob
import pickle

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

# Retry configuration
httplib2.RETRIES = 1
MAX_RETRIES = 10
RETRIABLE_EXCEPTIONS = (
    httplib2.HttpLib2Error,
    IOError,
    httplib.NotConnected,
    httplib.IncompleteRead,
    httplib.ImproperConnectionState,
    httplib.CannotSendRequest,
    httplib.CannotSendHeader,
    httplib.ResponseNotReady,
    httplib.BadStatusLine,
)
RETRIABLE_STATUS_CODES = [500, 502, 503, 504]

# OAuth 2.0 configuration
CLIENT_SECRETS_FILE = "client_secrets.json"
YOUTUBE_UPLOAD_SCOPE = "https://www.googleapis.com/auth/youtube.upload"
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
MISSING_CLIENT_SECRETS_MESSAGE = """
WARNING: Please configure OAuth 2.0
To make this sample run you will need to populate the client_secrets.json file
found at:
   %s
with information from the API Console
https://console.cloud.google.com/
For more information about the client_secrets.json file format, please visit:
https://developers.google.com/api-client-library/python/guide/aaa_client_secrets
"""
VALID_PRIVACY_STATUSES = ("unlisted", "public", "private")

def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(
        CLIENT_SECRETS_FILE, scopes=[YOUTUBE_UPLOAD_SCOPE]
    )

    credentials = None

    # Check if token file exists
    if os.path.exists("token.pickle"):
        # If it does, load the credentials from it
        try:
            with open("token.pickle", "rb") as token:
                credentials = pickle.load(token)
        except EOFError:
            # If an EOFError was raised, the file is empty. Credentials will be None.
            pass

    # If there are no valid credentials available, prompt the user to log in
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            credentials = flow.run_local_server(port=0)

        # Save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(credentials, token)

    return build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, credentials=credentials)

def initialize_upload(youtube, options):
    print("Initializing Upload")
    tags = None
    if options.keywords:
        tags = options.keywords.split(",")

    body = dict(
        snippet=dict(
            title=options.title,
            description=options.description,
            tags=tags,
            categoryId=options.category,
        ),
        status=dict(privacyStatus=options.privacyStatus),
    )

    # Call the API's videos.insert method to create and upload the video.
    insert_request = youtube.videos().insert(
        part=",".join(body.keys()),
        body=body,
        media_body=MediaFileUpload(options.file, chunksize=-1, resumable=True),
    )

    resumable_upload(insert_request)

def resumable_upload(insert_request):
    response = None
    error = None
    retry = 0
    while response is None:
        try:
            print("Uploading file...")
            status, response = insert_request.next_chunk()
            if response is not None:
                if "id" in response:
                    print("Video id '%s' was successfully uploaded." % response["id"])
                else:
                    exit("The upload failed with an unexpected response: %s" % response)
        except HttpError as e:
            if e.resp.status in RETRIABLE_STATUS_CODES:
                error = "A retriable HTTP error %d occurred:\n%s" % (
                    e.resp.status,
                    e.content,
                )
            else:
                raise
        except RETRIABLE_EXCEPTIONS as e:
            error = "A retriable error occurred: %s" % e

        if error is not None:
            print(error)
            retry += 1
            if retry > MAX_RETRIES:
                exit("No longer attempting to retry.")

            max_sleep = 2**retry
            sleep_seconds = random.random() * max_sleep
            print("Sleeping %f seconds and then retrying..." % sleep_seconds)
            time.sleep(sleep_seconds)

if __name__ == "__main__":
    # Create an ArgumentParser object
    argparser = argparse.ArgumentParser()

    # Add the necessary arguments
    # argparser.add_argument("--file", required=True, help="Video file to upload")
    argparser.add_argument("--title", help="Video title", default="Test Title")
    argparser.add_argument(
        "--description",
        help="Video description",
        default="Uploading automatically using code",
    )
    argparser.add_argument(
        "--category",
        default="28",
        help="Numeric video category. "
        "See https://developers.google.com/youtube/v3/docs/videoCategories/list",
    )
    argparser.add_argument(
        "--keywords", help="Video keywords, comma separated", default=""
    )
    argparser.add_argument(
        "--privacyStatus",
        choices=VALID_PRIVACY_STATUSES,
        default=VALID_PRIVACY_STATUSES[0],
        help="Video privacy status.",
    )
    args = argparser.parse_args()

    youtube = get_authenticated_service()

    # Get a list of all video files in the "videos" folder
    video_files = (
        glob.glob("videos/*.mp4")
        + glob.glob("videos/*.avi")
        + glob.glob("videos/*.mov")
        + glob.glob("videos/*.flv")
        + glob.glob("videos/*.wmv")
        + glob.glob("videos/*.mkv")
        + glob.glob("videos/*.webm")
        + glob.glob("videos/*.mpeg")
        + glob.glob("videos/*.mpg")
        + glob.glob("videos/*.m4v")
        + glob.glob("videos/*.3gp")
    )

    # Loop over the list of video files and upload each one
    for video_file in video_files:
        if not os.path.exists(video_file):
            print(f"The file {video_file} does not exist.")
            continue

        args.file = video_file
        # If no title argument is specified, use the filename as the title
        if args.title == "Test Title":
            args.title = os.path.basename(video_file)
        print(f"Uploading {video_file}...")
        try:
            initialize_upload(youtube, args)
        except HttpError as e:
            print("An HTTP error %d occurred:\n%s" % (e.resp.status, e.content))
